# 加载库函数

In [1]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
import networkx.algorithms.isomorphism as iso
import itertools
import os.path
import subprocess
import torch.nn.functional as F
import pickle
import os
import scipy.sparse as sp
import torch
import json
import scipy.io as sio
from torch.nn import Embedding
#import scipy.sparse as spsparse
from torch.nn import Parameter
import torch
import torch.nn as nn
from scipy.sparse import csr_matrix
import torch.nn.functional as F
import argparse
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

# 加载数据，数据处理

In [2]:
"""
Disclaimer: functions defined from lines 15 to 36 in this file come from 
tkipf/gae original repository on Graph Autoencoders. Moreover, the
mask_test_edges_general_link_prediction function is borrowed from 
philipjackson's mask_test_edges pull request on this same repository.
"""

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

# 任务一

In [3]:
def mask_test_edges_general_link_prediction(adj, test_percent=10., val_percent=5.):
    """
    Task 1: General Directed Link Prediction: get Train/Validation/Test

    :param adj: complete sparse adjacency matrix of the graph
    :param test_percent: percentage of edges in test set
    :param val_percent: percentage of edges in validation set
    :return: train incomplete adjacency matrix, validation and test sets
    """

    # Remove diagonal elements of adjacency matrix
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    edges_positive, _, _ = sparse_to_tuple(adj)

    # Number of positive (and negative) edges in test and val sets:
    num_test = int(np.floor(edges_positive.shape[0] / (100. / test_percent)))
    num_val = int(np.floor(edges_positive.shape[0] / (100. / val_percent)))

    # Sample positive edges for test and val sets:
    edges_positive_idx = np.arange(edges_positive.shape[0])
    np.random.shuffle(edges_positive_idx)
    val_edge_idx = edges_positive_idx[:num_val]
    # positive val edges
    val_edges = edges_positive[val_edge_idx]
    test_edge_idx = edges_positive_idx[num_val:(num_val + num_test)]
    # positive test edges
    test_edges = edges_positive[test_edge_idx]
    # positive train edges
    train_edges = np.delete(edges_positive, np.hstack([test_edge_idx, val_edge_idx]), axis = 0)

    # (Text from philipjackson)
    # The above strategy for sampling without replacement will not work for sampling
    # negative edges on large graphs, because the pool of negative edges
    # is much much larger due to sparsity, therefore we'll use the following strategy:
    # 1. sample random linear indices from adjacency matrix WITH REPLACEMENT
    # (without replacement is super slow). sample more than we need so we'll probably
    # have enough after all the filtering steps.
    # 2. remove any edges that have already been added to the other edge lists
    # 3. convert to (i,j) coordinates
    # 4. remove any duplicate elements if there are any
    # 5. remove any diagonal elements
    # 6. if we don't have enough edges, repeat this process until we get enough
    positive_idx, _, _ = sparse_to_tuple(adj) # [i,j] coord pairs for all true edges
    positive_idx = positive_idx[:,0]*adj.shape[0] + positive_idx[:,1] # linear indices
    # Test set
    test_edges_false = np.empty((0,2),dtype='int64')
    idx_test_edges_false = np.empty((0,),dtype='int64')
    while len(test_edges_false) < len(test_edges):
        # step 1:
        idx = np.random.choice(adj.shape[0]**2, 2*(num_test - len(test_edges_false)), replace = True)
        # step 2:
        idx = idx[~np.in1d(idx, positive_idx, assume_unique = True)]
        idx = idx[~np.in1d(idx, idx_test_edges_false, assume_unique = True)]
        # step 3:
        rowidx = idx // adj.shape[0]
        colidx = idx % adj.shape[0]
        coords = np.vstack((rowidx, colidx)).transpose()
        # step 4:
        coords = np.unique(coords, axis=0)
        np.random.shuffle(coords)
        # step 5:
        coords = coords[coords[:,0] != coords[:,1]]
        # step 6:
        coords = coords[:min(num_test, len(idx))]
        test_edges_false = np.append(test_edges_false, coords, axis = 0)
        idx = idx[:min(num_test, len(idx))]
        idx_test_edges_false = np.append(idx_test_edges_false, idx)

    # Validation set
    val_edges_false = np.empty((0,2), dtype = 'int64')
    idx_val_edges_false = np.empty((0,), dtype = 'int64')
    while len(val_edges_false) < len(val_edges):
        # step 1:
        idx = np.random.choice(adj.shape[0]**2, 2*(num_val - len(val_edges_false)), replace = True)
        # step 2:
        idx = idx[~np.in1d(idx, positive_idx, assume_unique = True)]
        idx = idx[~np.in1d(idx, idx_test_edges_false, assume_unique = True)]
        idx = idx[~np.in1d(idx, idx_val_edges_false, assume_unique = True)]
        # step 3:
        rowidx = idx // adj.shape[0]
        colidx = idx % adj.shape[0]
        coords = np.vstack((rowidx, colidx)).transpose()
        # step 4:
        coords = np.unique(coords, axis = 0)
        np.random.shuffle(coords)
        # step 5:
        coords = coords[coords[:,0] != coords[:,1]]
        # step 6:
        coords = coords[:min(num_val, len(idx))]
        val_edges_false = np.append(val_edges_false, coords, axis=0)
        idx = idx[:min(num_val, len(idx))]
        idx_val_edges_false = np.append(idx_val_edges_false, idx)

    # Sanity checks:
    train_edges_linear = train_edges[:,0]*adj.shape[0] + train_edges[:,1]
    test_edges_linear = test_edges[:,0]*adj.shape[0] + test_edges[:,1]
    assert not np.any(np.in1d(idx_test_edges_false, positive_idx))
    assert not np.any(np.in1d(idx_val_edges_false, positive_idx))
    assert not np.any(np.in1d(val_edges[:,0]*adj.shape[0] + val_edges[:,1], train_edges_linear))
    assert not np.any(np.in1d(test_edges_linear, train_edges_linear))
    assert not np.any(np.in1d(val_edges[:,0]*adj.shape[0] + val_edges[:,1], test_edges_linear))

    # Re-build train adjacency matrix
    #data = np.ones(train_edges.shape[0])
    #adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape = adj.shape)

    return train_edges, val_edges, val_edges_false, test_edges, test_edges_false

# 任务二

In [4]:
def mask_test_edges_biased_negative_samples(adj, test_percent=10.):
    """
    Task 2: General Biased Negative Samples (B.N.S.) Directed Link
    Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :param test_percent: percentage of edges in test set
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements of adjacency matrix
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Retrieve all unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()
    edges_positive, _, _ = sparse_to_tuple(adj_tilde)

    # Number of positive (= to number of negative) test node pairs to sample
    num_test = int(np.floor(edges_positive.shape[0] / (100. / test_percent)))

    # Sampling of positive node pairs
    edges_idx = np.arange(edges_positive.shape[0])
    np.random.shuffle(edges_idx)
    test_edges_idx = edges_idx[:num_test]
    test_edges = edges_positive[test_edges_idx]

    # In this setting, the reverse node pairs constitute negative samples
    test_edges_false = np.fliplr(test_edges)

    # Get training incomplete adjacency matrix
    train_edges = np.delete(edges_positive, np.hstack([test_edges_idx]), axis = 0)
    #data = np.ones(train_edges.shape[0])
    #adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape = adj.shape)

    # Validation set: not implemented for Task 2
    # therefore, val_edges and val_edges_false are None
    return train_edges, val_edges, val_edges_false, test_edges, test_edges_false

# 任务三

In [5]:
# Task 3 - Bidirectional Link Prediction
def mask_test_edges_bidirectional_link_prediction(adj):
    """
    Task 3: Bidirectionality Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()

    ## Bidirectional edges (they usually are few, so they are all in test set)
    adj_sym_ones = adj - adj_tilde
    adj_sym_ones.eliminate_zeros()

    # Positive node pairs in test set (bidirectional edges)
    test_edges, _, _ = sparse_to_tuple(adj_sym_ones)
    test_edges = test_edges[test_edges[:,1] > test_edges[:,0],:]

    # Negative node pairs in test set (unidirectional edges)
    test_edges_false, _, _ = sparse_to_tuple(adj_tilde)
    test_edges_false = test_edges_false[test_edges_false[:,0] > test_edges_false[:,1],:]
    test_edges_false = np.fliplr(test_edges_false)
    # Sampling of negative node pairs
    edges_negative_idx = np.arange(test_edges_false.shape[0])
    np.random.shuffle(edges_negative_idx)
    test_edges_false_idx = edges_negative_idx[:test_edges.shape[0]]
    test_edges_false = test_edges_false[test_edges_false_idx]

    # Get training incomplete adjacency matrix
    # 1 of the 2 directions of each bidirectional edge is masked
    adj_train = (adj - sp.triu(adj_sym_ones))

    # Validation set: not implemented for Task 2
    # val_edges and val_edges_false are None
    return adj_train, val_edges, val_edges_false, test_edges, test_edges_false

In [6]:
def load_data(dataset):
    """ Load datasets from text files

    :param dataset: 'cora', 'citeseer' or 'google' graph dataset.
    :return: n*n sparse adjacency matrix and n*f node-level feature matrix

    Note: in this paper, all three datasets are assumed to be featureless.
    As a consequence, the feature matrix is the identity matrix I_n.
    """
    if dataset == 'cora':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/cora.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Cora raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'citeseer':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/citeseer.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Citeseer raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'google':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/GoogleNw.txt",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        features = sp.identity(adj.shape[0])

    else:
        raise ValueError('Undefined dataset!')

    return adj, features

In [7]:
def get_graph(DATASET='cora', task=1):
    adj_init, features = load_data(DATASET)
    if task==1:
        train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_general_link_prediction(adj_init, 10, 5)
    elif task==2:
        train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_biased_negative_samples(adj_init)
    else:
        train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_bidirectional_link_prediction(adj_init)
    
    all_negative_edges = np.argwhere(adj_init.toarray()==0)
    
    if task==2 or task==3:
        all_train_edges_false = np.delete(all_negative_edges, test_edges_false, axis = 0)
    else:
        all_train_edges_false = np.delete(all_negative_edges, np.vstack([test_edges_false, val_edges_false]), axis = 0)
    
    if DATASET=='cora':
        nodes_number = 2708
    elif DATASET=='citeseer':
        nodes_number = 3327
    
    ################## 处理测试集  #############################
    test_posi_mask = [test_edges[i][0]*nodes_number+test_edges[i][1] for i in range(len(test_edges))]
    test_false_mask = [test_edges_false[i][0]*nodes_number+test_edges_false[i][1] for i in range(len(test_edges_false))]
    test_mask = test_posi_mask + test_false_mask
    test_mask = torch.tensor(test_mask)
    test_edge_true_or_false = torch.zeros(len(test_mask))
    test_edge_true_or_false[:len(test_posi_mask)] = 1
    
    ############### 处理训练集  ################################
    negative_index = np.random.choice(range(all_train_edges_false.shape[0]), train_edges.shape[0], replace=False)
    train_edges_false = all_train_edges_false[negative_index]
    train_posi_mask = [train_edges[i][0]*nodes_number+train_edges[i][1] for i in range(len(train_edges))]
    train_false_mask = [train_edges_false[i][0]*nodes_number+train_edges_false[i][1] for i in range(len(train_edges_false))]
    train_mask = train_posi_mask + train_false_mask
    train_mask = torch.tensor(train_mask)
    #train_mask = torch.tensor(train_posi_mask)
    train_edge_true_or_false = torch.zeros(len(train_mask))
    train_edge_true_or_false[:len(train_posi_mask)] = 1
    
    return train_edge_true_or_false, test_edge_true_or_false, train_mask.long(), test_mask.long(), nodes_number

In [8]:
#train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(task=2)

# 定义模型

In [9]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    ap = average_precision_score(true_edges, predict_edges)
    auc = roc_auc_score(true_edges, predict_edges)
    print("AP SCORE： ", ap)
    print("AUC SCORE: ", auc)
    return ap, auc

################  更新节点内容  #######################################
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, context_embedding):
        super(reconstruction_graph, self).__init__()
        self.context_embedding = Parameter(context_embedding)
        
    def forward(self, importance_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.context_embedding[:, None]-self.context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((self.context_embedding.unsqueeze(1) - importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        #return 2*sigmoid_modle(nodes_similar)-1
        return torch.exp(-nodes_similar)
        
    
class update_nodes_embedding(nn.Module):
    def __init__(self, context_embedding):
        super(update_nodes_embedding, self).__init__()
        self.reconstruction_module = reconstruction_graph(context_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, importance_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(importance_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.context_embedding.detach(), ap, auc

####################  更新节点重要性  ######################################
class reconstruction_graph_importance(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, importance_embedding):
        super(reconstruction_graph_importance, self).__init__()
        self.importance_embedding = Parameter(importance_embedding)
        
    def forward(self, context_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(context_embedding[:, None]-context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((context_embedding.unsqueeze(1) - self.importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        #return 2*sigmoid_modle(nodes_similar)-1
        return torch.exp(-nodes_similar)

class update_nodes_embedding_importance(nn.Module):
    def __init__(self, importance_embedding):
        super(update_nodes_embedding_importance, self).__init__()
        self.reconstruction_module = reconstruction_graph_importance(importance_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, context_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(context_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.importance_embedding.detach(), ap, auc

In [10]:
context_embedding_dim = 12
importance_embedding_dim = 48
beta = 5.0
ALPHA = 1.0
LR = 0.01
MOMENTUM = 0.9
EPOCH = 5
N = 25.0    # 目前25最佳
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'cora'

In [11]:
train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset, task=2)
context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
nodes_context_embeddings = context_embedding.weight.detach()

importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
nodes_importance_embeddings = importance_embedding.weight.detach()
    
auc_first_best_auc = 0
auc_first_best_ap = 0
ap_first_best_auc = 0
ap_first_best_ap = 0

for module_epoch in range(100):
    print("######################### 模块循环 ： %d ##########################"%module_epoch)
    update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
    nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
    if auc > auc_first_best_auc:
        auc_first_best_auc = auc
        auc_first_best_ap = ap
    if ap > ap_first_best_ap:
        ap_first_best_auc = auc
        ap_first_best_ap = ap
    update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
    nodes_importance_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
    if auc > auc_first_best_auc:
        auc_first_best_auc = auc
        auc_first_best_ap = ap
    if ap > ap_first_best_ap:
        ap_first_best_auc = auc
        ap_first_best_ap = ap

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 2453.3003
Epoch: 01, Loss: 2442.0999
Epoch: 02, Loss: 2421.7202
Epoch: 03, Loss: 2394.8330
Epoch: 04, Loss: 2364.2095
AP SCORE：  0.5224825737713961
AUC SCORE:  0.5347061157226562
Epoch: 00, Loss: 2312.9524
Epoch: 01, Loss: 2282.5803
Epoch: 02, Loss: 2252.8315
Epoch: 03, Loss: 2223.9336
Epoch: 04, Loss: 2195.9570
AP SCORE：  0.6043222231261123
AUC SCORE:  0.5875587463378906
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 2168.9048
Epoch: 01, Loss: 2165.6868
Epoch: 02, Loss: 2159.6211
Epoch: 03, Loss: 2151.0808
Epoch: 04, Loss: 2140.4272
AP SCORE：  0.6312418641043818
AUC SCORE:  0.6079940795898438
Epoch: 00, Loss: 2128.0027
Epoch: 01, Loss: 2114.1240
Epoch: 02, Loss: 2099.0789
Epoch: 03, Loss: 2083.1208
Epoch: 04, Loss: 2066.4731
AP SCORE：  0.6638308060209772
AUC SCORE:  0.6359024047851562
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 2049.3264
Epo

Epoch: 00, Loss: 1269.0023
Epoch: 01, Loss: 1268.3499
Epoch: 02, Loss: 1267.1115
Epoch: 03, Loss: 1265.3496
Epoch: 04, Loss: 1263.1199
AP SCORE：  0.8315334334460569
AUC SCORE:  0.7968215942382812
Epoch: 00, Loss: 1260.4751
Epoch: 01, Loss: 1257.4619
Epoch: 02, Loss: 1254.1241
Epoch: 03, Loss: 1250.5015
Epoch: 04, Loss: 1246.6299
AP SCORE：  0.8328755620457913
AUC SCORE:  0.7983932495117188
######################### 模块循环 ： 19 ##########################
Epoch: 00, Loss: 1242.5420
Epoch: 01, Loss: 1241.9280
Epoch: 02, Loss: 1240.7632
Epoch: 03, Loss: 1239.1053
Epoch: 04, Loss: 1237.0079
AP SCORE：  0.8336184887004681
AUC SCORE:  0.7992439270019531
Epoch: 00, Loss: 1234.5198
Epoch: 01, Loss: 1231.6854
Epoch: 02, Loss: 1228.5461
Epoch: 03, Loss: 1225.1388
Epoch: 04, Loss: 1221.4974
AP SCORE：  0.8347336744652346
AUC SCORE:  0.800628662109375
######################### 模块循环 ： 20 ##########################
Epoch: 00, Loss: 1217.6532
Epoch: 01, Loss: 1217.0759
Epoch: 02, Loss: 1215.9806
Epoch: 03,

Epoch: 02, Loss: 967.7139
Epoch: 03, Loss: 967.0844
Epoch: 04, Loss: 966.2876
AP SCORE：  0.8520609077753885
AUC SCORE:  0.8236312866210938
Epoch: 00, Loss: 965.3418
Epoch: 01, Loss: 964.2637
Epoch: 02, Loss: 963.0682
Epoch: 03, Loss: 961.7696
Epoch: 04, Loss: 960.3802
AP SCORE：  0.8524087086108008
AUC SCORE:  0.8242034912109375
######################### 模块循环 ： 37 ##########################
Epoch: 00, Loss: 958.9115
Epoch: 01, Loss: 958.6899
Epoch: 02, Loss: 958.2695
Epoch: 03, Loss: 957.6711
Epoch: 04, Loss: 956.9135
AP SCORE：  0.8525037467040943
AUC SCORE:  0.8244132995605469
Epoch: 00, Loss: 956.0141
Epoch: 01, Loss: 954.9889
Epoch: 02, Loss: 953.8523
Epoch: 03, Loss: 952.6172
Epoch: 04, Loss: 951.2959
AP SCORE：  0.8527601180420041
AUC SCORE:  0.8248062133789062
######################### 模块循环 ： 38 ##########################
Epoch: 00, Loss: 949.8989
Epoch: 01, Loss: 949.6882
Epoch: 02, Loss: 949.2883
Epoch: 03, Loss: 948.7188
Epoch: 04, Loss: 947.9980
AP SCORE：  0.8528461185001344
AU

Epoch: 03, Loss: 846.0232
Epoch: 04, Loss: 845.3693
AP SCORE：  0.8551078749688762
AUC SCORE:  0.8308944702148438
######################### 模块循环 ： 55 ##########################
Epoch: 00, Loss: 844.6769
Epoch: 01, Loss: 844.5719
Epoch: 02, Loss: 844.3724
Epoch: 03, Loss: 844.0884
Epoch: 04, Loss: 843.7286
AP SCORE：  0.8550699566171647
AUC SCORE:  0.8309745788574219
Epoch: 00, Loss: 843.3011
Epoch: 01, Loss: 842.8132
Epoch: 02, Loss: 842.2714
Epoch: 03, Loss: 841.6821
Epoch: 04, Loss: 841.0504
AP SCORE：  0.8551702161409731
AUC SCORE:  0.8311309814453125
######################### 模块循环 ： 56 ##########################
Epoch: 00, Loss: 840.3813
Epoch: 01, Loss: 840.2797
Epoch: 02, Loss: 840.0870
Epoch: 03, Loss: 839.8126
Epoch: 04, Loss: 839.4649
AP SCORE：  0.85516418749633
AUC SCORE:  0.8311958312988281
Epoch: 00, Loss: 839.0516
Epoch: 01, Loss: 838.5800
Epoch: 02, Loss: 838.0564
Epoch: 03, Loss: 837.4866
Epoch: 04, Loss: 836.8759
AP SCORE：  0.8551490536247648
AUC SCORE:  0.8312644958496094

Epoch: 02, Loss: 784.5013
Epoch: 03, Loss: 784.3358
Epoch: 04, Loss: 784.1259
AP SCORE：  0.8547883097954772
AUC SCORE:  0.8324089050292969
Epoch: 00, Loss: 783.8763
Epoch: 01, Loss: 783.5913
Epoch: 02, Loss: 783.2747
Epoch: 03, Loss: 782.9299
Epoch: 04, Loss: 782.5602
AP SCORE：  0.8547193574184402
AUC SCORE:  0.8323631286621094
######################### 模块循环 ： 74 ##########################
Epoch: 00, Loss: 782.1681
Epoch: 01, Loss: 782.1086
Epoch: 02, Loss: 781.9952
Epoch: 03, Loss: 781.8338
Epoch: 04, Loss: 781.6293
AP SCORE：  0.8546999705602405
AUC SCORE:  0.832366943359375
Epoch: 00, Loss: 781.3860
Epoch: 01, Loss: 781.1083
Epoch: 02, Loss: 780.7997
Epoch: 03, Loss: 780.4637
Epoch: 04, Loss: 780.1033
AP SCORE：  0.8546223315134249
AUC SCORE:  0.8323478698730469
######################### 模块循环 ： 75 ##########################
Epoch: 00, Loss: 779.7211
Epoch: 01, Loss: 779.6629
Epoch: 02, Loss: 779.5524
Epoch: 03, Loss: 779.3951
Epoch: 04, Loss: 779.1957
AP SCORE：  0.8546193996520648
AUC

Epoch: 03, Loss: 746.1143
Epoch: 04, Loss: 745.8638
AP SCORE：  0.8539590024174221
AUC SCORE:  0.832244873046875
######################### 模块循环 ： 92 ##########################
Epoch: 00, Loss: 745.5982
Epoch: 01, Loss: 745.5577
Epoch: 02, Loss: 745.4808
Epoch: 03, Loss: 745.3711
Epoch: 04, Loss: 745.2321
AP SCORE：  0.8539553831366442
AUC SCORE:  0.8322525024414062
Epoch: 00, Loss: 745.0667
Epoch: 01, Loss: 744.8779
Epoch: 02, Loss: 744.6678
Epoch: 03, Loss: 744.4389
Epoch: 04, Loss: 744.1934
AP SCORE：  0.8539362195532935
AUC SCORE:  0.8322296142578125
######################### 模块循环 ： 93 ##########################
Epoch: 00, Loss: 743.9327
Epoch: 01, Loss: 743.8931
Epoch: 02, Loss: 743.8176
Epoch: 03, Loss: 743.7100
Epoch: 04, Loss: 743.5735
AP SCORE：  0.8539181285283118
AUC SCORE:  0.8322067260742188
Epoch: 00, Loss: 743.4112
Epoch: 01, Loss: 743.2259
Epoch: 02, Loss: 743.0198
Epoch: 03, Loss: 742.7953
Epoch: 04, Loss: 742.5542
AP SCORE：  0.8539077567558336
AUC SCORE:  0.832241058349609

In [12]:
print("AUC FIRST BEST AUC: ", auc_first_best_auc)
print("AUC FIRST BEST AP: ", auc_first_best_ap)
print("AP FIRST BEST AUC: ", ap_first_best_auc)
print("AP FIRST BEST AP: ", ap_first_best_ap)

AUC FIRST BEST AUC:  0.8324203491210938
AUC FIRST BEST AP:  0.854598748638586
AP FIRST BEST AUC:  0.8311309814453125
AP FIRST BEST AP:  0.8551702161409731
